<a href="https://colab.research.google.com/github/Vetu2910/PCA/blob/main/coviddetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING THE DEPENDENCIES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras 
import tensorflow as tf

from keras.layers import Conv2D,Dense,MaxPool2D,Dropout,Flatten
from keras.models import Sequential
from keras.preprocessing import image

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Image augmentation to improve the performance

In [ ]:
train_datagen = image.ImageDataGenerator(
    rescale = 1/255,horizontal_flip = True,zoom_range = 0.2,shear_range = 0.2
)
train_data = train_datagen.flow_from_directory(directory = "/content/drive/MyDrive/project dataset complete",target_size = (256,256),batch_size = 16,class_mode = 'binary')

Found 13839 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'covid': 0, 'normal': 1}

In [ ]:
test_datagen = image.ImageDataGenerator(
    rescale = 1/255
)
test_data = train_datagen.flow_from_directory(directory = "/content/drive/MyDrive/project test",target_size = (256,256),batch_size = 16,class_mode = 'binary')

Found 72 images belonging to 2 classes.


In [ ]:
test_data.class_indices


{'covid': 0, 'normal': 1}

BUILDING OUR COVID MODEL

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',input_shape = (256,256,3)))

model.add(Conv2D(filters = 64,kernel_size = (3,3),activation = 'relu'))
model.add(MaxPool2D())
model.add(Dropout(rate = 0.25))

model.add(Conv2D(filters = 64,kernel_size = (3,3),activation = 'relu'))
model.add(MaxPool2D())
model.add(Dropout(rate = 0.25))

model.add(Conv2D(filters = 128,kernel_size = (3,3),activation = 'relu'))
model.add(MaxPool2D())
model.add(Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(units=64,activation = 'relu'))
model.add(Dropout(rate = 0.15))

model.add(Dense(units = 1,activation= 'softmax'))

model.compile(loss = keras.losses.BinaryCrossentropy(),optimizer = 'adam',metrics = ['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 126, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                    

In [ ]:
model.fit(train_data,# batch_size = 512,
          steps_per_epoch = 15  ,epochs = 50   # ,validation_split=0.2
                #   validation_data = test_data
                   )

Epoch 1/50
15/15 [==============================] - 79s 3s/step - loss: 0.7730 - acc: 0.7167
Epoch 2/50
15/15 [==============================] - 48s 3s/step - loss: 0.6201 - acc: 0.7167
Epoch 3/50
15/15 [==============================] - 51s 3s/step - loss: 0.6410 - acc: 0.6792
Epoch 4/50
15/15 [==============================] - 48s 3s/step - loss: 0.5992 - acc: 0.7333
Epoch 5/50
15/15 [==============================] - 50s 3s/step - loss: 0.5957 - acc: 0.7322
Epoch 6/50
15/15 [==============================] - 46s 3s/step - loss: 0.5596 - acc: 0.7792
Epoch 7/50
15/15 [==============================] - 51s 3s/step - loss: 0.6022 - acc: 0.7000
Epoch 8/50
15/15 [==============================] - 44s 3s/step - loss: 0.5740 - acc: 0.7583
Epoch 9/50
15/15 [==============================] - 46s 3s/step - loss: 0.5485 - acc: 0.7500
Epoch 10/50
15/15 [==============================] - 46s 3s/step - loss: 0.5265 - acc: 0.7583
Epoch 11/50
15/15 [==============================] - 45s 3s/step - lo

In [ ]:
# model.fit(test_data ,steps_per_epoch = 2,epochs = 50)

In [ ]:
model.compile(
    optimizer= tf.keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
results = model.evaluate(test_data,batch_size = 2)
print("test loss, test acc:", results)

5/5 [==============================] - 14s 3s/step - loss: nan - sparse_categorical_accuracy: 0.5833
test loss, test acc: [nan, 0.5833333134651184]


In [ ]:
path = "/content/drive/MyDrive/Chest.jpeg"
img = image.load_img(path,target_size=(256,256))

img = image.img_to_array(img)/255
img = np.array([img])
# tf.image.resize(img, [1,256,256,3]).shape.as_list()
img.shape
# img = image.reshape(1,256,256,3)

(1, 256, 256, 3)

In [ ]:
model.predict(img)
#(model.predict(img) > 0).astype("int32")
predictions = (model.predict(img) < 0.6).astype("int32")
predictions


array([[0]], dtype=int32)

class 0 --> covid affected
class 1 --> normal lung

In [ ]:
model.predict(img)

array([[0.7374685]], dtype=float32)

In [ ]:
path = "/content/drive/MyDrive/Normal.png"
img = image.load_img(path,target_size=(256,256))

img = image.img_to_array(img)/255
img = np.array([img])
# tf.image.resize(img, [1,256,256,3]).shape.as_list()
img.shape
# img = image.reshape(1,256,256,3)


(1, 256, 256, 3)

In [ ]:
# y = keras.np_utils.probas_to_classes(img)
#model.predict_classes(img)
(model.predict(img) > 0.6).astype("int32")

array([[1]], dtype=int32)